# 03 - DQ Exploration (Before vs After ETL)
# Goal - Compare reports/dq_before.json vs dq_after.json
# No data mutation, only reporting/visuals

In [13]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
import os

before_path = Path("../reports/dq_before.json")
after_path = Path("../reports/dq_after.json")

### Load JSON Reports

In [34]:
with open(before_path, 'r') as f:
    dq_before = json.load(f)

with open(after_path, 'r') as f:
    dq_after = json.load(f)
   
print("DQ_BEFORE KEYS: ", dq_before.keys())
print("DQ_AFTER KEYS: ", dq_after.keys())


DQ_BEFORE KEYS:  dict_keys(['meta', 'dataset', 'missingness', 'duplicates', 'numerical_anomalies', 'categorical_anomalies', 'logical_rules_anomalies', 'label_distribution'])
DQ_AFTER KEYS:  dict_keys(['meta', 'dataset', 'missingness', 'duplicates', 'numerical_anomalies', 'categorical_anomalies', 'logical_rules_anomalies', 'label_distribution'])


### Headline DQ Summary (Before vs After ETL)

This table highlights the most important data quality indicators to quickly
validate whether the ETL pipeline improved the dataset.

In [38]:
# Helper to safely extract nested values
def safe_get(d, path, default=None):
    for p in path:
        if isinstance(d, dict) and p in d:
            d = d[p]
        else:
            return default
    return d


summary_rows = []

# Dataset shape
summary_rows.append({
    "metric": "rows",
    "before": dq_before["dataset"]["shape"]["rows"],
    "after": dq_after["dataset"]["shape"]["rows"]
})

summary_rows.append({
    "metric": "columns",
    "before": dq_before["dataset"]["shape"]["cols"],
    "after": dq_after["dataset"]["shape"]["cols"]
})

# Missingness
summary_rows.append({
    "metric": "total_missing_values",
    "before": dq_before["missingness"]["total_missing"],
    "after": dq_after["missingness"]["total_missing"]
})

# Duplicates
summary_rows.append({
    "metric": "duplicate_rows",
    "before": dq_before["duplicates"]["duplicate_row_count"],
    "after": dq_after["duplicates"]["duplicate_row_count"]
})

# Logical rule violations (sum across rules)
summary_rows.append({
    "metric": "logical_rule_violations_total",
    "before": sum(dq_before["logical_rules_anomalies"]["violations_by_rule"].values()),
    "after": sum(dq_after["logical_rules_anomalies"]["violations_by_rule"].values())
})

# Numeric max values
for col in ["tenure", "MonthlyCharges", "TotalCharges"]:
    summary_rows.append({
        "metric": f"{col}_max",
        "before": dq_before["numerical_anomalies"]["per_column"][col]["max_value"],
        "after": dq_after["numerical_anomalies"]["per_column"][col]["max_value"]
    })

# Label distribution
summary_rows.append({
    "metric": "churn_yes_pct",
    "before": dq_before["label_distribution"]["pct"].get("Yes"),
    "after": dq_after["label_distribution"]["pct"].get("Yes")
})

summary_rows.append({
    "metric": "churn_nan_count",
    "before": dq_before["label_distribution"]["nan_count"],
    "after": dq_after["label_distribution"]["nan_count"]
})

summary_df = pd.DataFrame(summary_rows).set_index("metric")
summary_df

,before,after
metric,,
rows,8099.000000,8099.00000
columns,21.000000,23.00000
total_missing_values,3240.000000,0.00000
duplicate_rows,2112.000000,2112.00000
logical_rule_violations_total,1109.000000,0.00000
tenure_max,1000000.000000,72.00000
MonthlyCharges_max,1000000.000000,200.00000
TotalCharges_max,1000000.000000,20000.00000
churn_yes_pct,0.265094,0.27917


### Missingness comparison

In [45]:
# --- Missingness per column (before vs after) ---
missing_before = dq_before["missingness"]["missing_by_column"]
missing_after = dq_after["missingness"]["missing_by_column"]

# Convert to Series (handles if they are already dict-like)
s_before = pd.Series(missing_before, name="missing_before").astype(int)
s_after = pd.Series(missing_after, name="missing_after").astype(int)

# Align columns across both (in case one report has extra/missing columns)
missing_compare = pd.concat([s_before, s_after], axis=1).fillna(0).astype(int)
missing_compare["delta"] = missing_compare["missing_after"] - missing_compare["missing_before"]
missing_compare["improvement"] = -missing_compare["delta"]  # positive = reduced missingness

# Sort by most missing BEFORE (then by improvement)
missing_compare = missing_compare.sort_values(
    by=["missing_before", "improvement"],
    ascending=[False, False]
)

missing_compare[missing_compare["improvement"] > 0].head(20)

,missing_before,missing_after,delta,improvement
TotalCharges,573,0,-573,573
tenure,571,0,-571,571
MonthlyCharges,553,0,-553,553
OnlineBackup,113,0,-113,113
SeniorCitizen,107,0,-107,107
StreamingTV,106,0,-106,106
PhoneService,104,0,-104,104
Contract,103,0,-103,103
PaymentMethod,103,0,-103,103
PaperlessBilling,100,0,-100,100


### Numeric anomalies comparison

In [51]:
# --- Numeric anomalies comparison (before vs after) ---

num_cols = dq_after["numerical_anomalies"]["columns"]

before_num = dq_before["numerical_anomalies"]["per_column"]
after_num = dq_after["numerical_anomalies"]["per_column"]

rows = []
for col in num_cols:
    b = before_num.get(col, {})
    a = after_num.get(col, {})

    rows.append({
        "column": col,

        "nan_before": b.get("nan_count"),
        "nan_after":  a.get("nan_count"),
        "nan_delta":  (a.get("nan_count") - b.get("nan_count")) if (b.get("nan_count") is not None and a.get("nan_count") is not None) else None,

        "inf_before": b.get("inf_count"),
        "inf_after":  a.get("inf_count"),
        "inf_delta":  (a.get("inf_count") - b.get("inf_count")) if (b.get("inf_count") is not None and a.get("inf_count") is not None) else None,

        "neg_before": b.get("negative_count"),
        "neg_after":  a.get("negative_count"),
        "neg_delta":  (a.get("negative_count") - b.get("negative_count")) if (b.get("negative_count") is not None and a.get("negative_count") is not None) else None,

        "min_before": b.get("min_value"),
        "min_after":  a.get("min_value"),

        "median_before": b.get("median_value"),
        "median_after":  a.get("median_value"),

        "max_before": b.get("max_value"),
        "max_after":  a.get("max_value"),
    })

numeric_compare_df = pd.DataFrame(rows).set_index("column")
numeric_compare_df.sort_values(by="max_before", ascending=False)
numeric_compare_df

,nan_before,nan_after,nan_delta,inf_before,inf_after,inf_delta,neg_before,neg_after,neg_delta,min_before,min_after,median_before,median_after,max_before,max_after
column,,,,,,,,,,,,,,,
tenure,645,0,-645,24,0,-24,55,0,-55,-9999.0,0.0,28.00,29.00,1000000.0,72.0
MonthlyCharges,623,0,-623,20,0,-20,73,0,-73,-9999.0,0.0,70.15,70.40,1000000.0,200.0
TotalCharges,639,0,-639,27,0,-27,64,0,-64,-9999.0,0.0,1357.10,1389.85,1000000.0,20000.0


### Numeric Domain Enforcement Validation

Verify that post-ETL numeric values respect domain constraints:
- tenure ≤ 72
- MonthlyCharges ≤ 200
- TotalCharges ≤ 20000

In [52]:
domain_limits = {
    "tenure": 72,
    "MonthlyCharges": 200,
    "TotalCharges": 20000
}

rows = []
for col, limit in domain_limits.items():
    after_max = dq_after["numerical_anomalies"]["per_column"][col]["max_value"]
    rows.append({
        "column": col,
        "domain_max": limit,
        "observed_max_after": after_max,
        "passes": after_max <= limit
    })

pd.DataFrame(rows)


,column,domain_max,observed_max_after,passes
0,tenure,72,72.0,True
1,MonthlyCharges,200,200.0,True
2,TotalCharges,20000,20000.0,True


### Categorical anomalies summary

In [53]:
before_cat = dq_before["categorical_anomalies"]["per_column"]
after_cat = dq_after["categorical_anomalies"]["per_column"]

rows = []
for col in before_cat.keys():
    b = before_cat[col]
    a = after_cat.get(col, {})

    rows.append({
        "column": col,
        "nan_before": b.get("nan_count"),
        "nan_after": a.get("nan_count"),
        "whitespace_before": b.get("whitespace_count"),
        "whitespace_after": a.get("whitespace_count"),
        "unknown_before": b.get("unknown_values_count"),
        "unknown_after": a.get("unknown_values_count"),
        "invalid_before": b.get("invalid_category_count"),
        "invalid_after": a.get("invalid_category_count"),
    })

cat_compare_df = pd.DataFrame(rows).set_index("column")
cat_compare_df.sort_values("invalid_before", ascending=False).head(15)

,nan_before,nan_after,whitespace_before,whitespace_after,unknown_before,unknown_after,invalid_before,invalid_after
column,,,,,,,,
SeniorCitizen,107,0,0,0,262,0,7730,0
PhoneService,104,0,231,0,265,0,527,0
Dependents,93,0,223,0,276,0,523,0
DeviceProtection,75,0,224,0,294,0,523,0
Partner,97,0,223,0,272,0,522,0
StreamingMovies,83,0,221,0,286,0,522,0
PaymentMethod,103,0,228,0,266,0,522,0
StreamingTV,106,0,224,0,263,0,520,0
gender,99,0,226,0,270,0,519,0


### Invalid category excamples

In [54]:
interesting_cols = ["InternetService", "PaymentMethod", "MultipleLines"]

for col in interesting_cols:
    print(f"\n--- {col} ---")
    print("Before examples:", before_cat[col].get("invalid_examples", []))
    print("After examples:", after_cat[col].get("invalid_examples", []))


--- InternetService ---
Before examples: ['Fiber', 'DSL ', '0', ' DSL ', ' Fiber optic', ' DSL', 'No ', ' No ', ' No', 'N']
After examples: []

--- PaymentMethod ---
Before examples: ['Bank Transfer', 'Electronic Payment', 'Credit Card', 'Mailed check ', 'Credit card (automatic) ', ' Electronic check ', 'Mailed Check', ' Mailed check ', 'Paper Check', 'CC']
After examples: []

--- MultipleLines ---
Before examples: ['NPS', 'No ', 'y', '1', ' No', ' Yes ', '0', ' No ', 'n', 'no phone service']
After examples: []


### Logical rules violation comparison

In [58]:
before_rules = dq_before["logical_rules_anomalies"]["violations_by_rule"]
after_rules = dq_after["logical_rules_anomalies"]["violations_by_rule"]

rules_df = (
    pd.DataFrame({
        "violations_before": before_rules,
        "violations_after": after_rules
    })
    .fillna(0)
)

rules_df["delta"] = rules_df["violations_after"] - rules_df["violations_before"]
rules_df.sort_values("violations_before", ascending=False)


,violations_before,violations_after,delta
internet_dependency::StreamingTV,195,0.0,-195.0
internet_dependency::TechSupport,192,0.0,-192.0
internet_dependency::StreamingMovies,186,0.0,-186.0
internet_dependency::DeviceProtection,184,0.0,-184.0
internet_dependency::OnlineSecurity,179,0.0,-179.0
internet_dependency::OnlineBackup,173,0.0,-173.0


### Duplicate Metrics Comparison

In [59]:
dup_metrics = [
    "duplicate_row_count",
    "duplicate_id_count",
    "duplicate_flagged_correctly"
]

rows = []
for m in dup_metrics:
    rows.append({
        "metric": m,
        "before": dq_before["duplicates"].get(m),
        "after": dq_after["duplicates"].get(m),
    })

pd.DataFrame(rows).set_index("metric")

,before,after
metric,,
duplicate_row_count,2112.0,2112
duplicate_id_count,1056.0,1056
duplicate_flagged_correctly,NaN,True


### Label Distribution comparison

In [61]:
rows = []
for label in ["Yes", "No"]:
    rows.append({
        "label": label,
        "before_pct": dq_before["label_distribution"]["pct"].get(label),
        "after_pct": dq_after["label_distribution"]["pct"].get(label),
    })

rows.append({
    "label": "NaN",
    "before_pct": dq_before["label_distribution"]["nan_count"],
    "after_pct": dq_after["label_distribution"]["nan_count"],
})

pd.DataFrame(rows)

,label,before_pct,after_pct
0,Yes,0.265094,0.27917
1,No,0.691567,0.72083
2,NaN,0.000000,0.00000


### DQ Exploration Summary

Key observations:
- Total missing values were significantly reduced after ETL.
- All logical dependency violations were eliminated.
- Numeric corruption was repaired using domain-driven rules.
- Categorical inconsistencies, whitespace issues, and unknown tokens were normalized.
- Target label (`Churn`) distribution was preserved, avoiding label leakage.

This confirms the ETL pipeline improves data quality without distorting modeling signals.

In [67]:
dq_compare_summary = {
    "rows_before": dq_before["dataset"]["shape"]["rows"],
    "rows_after": dq_after["dataset"]["shape"]["rows"],
    "total_missing_before": dq_before["missingness"]["total_missing"],
    "total_missing_after": dq_after["missingness"]["total_missing"],
    "logical_violations_before": sum(before_rules.values()),
    "logical_violations_after": sum(after_rules.values()),
    "numeric_max_after": {
        col: dq_after["numerical_anomalies"]["per_column"][col]["max_value"]
        for col in domain_limits
    }
}

with open("../reports/dq_compare_summary.json", "w") as f:
    json.dump(dq_compare_summary, f, indent=2)

dq_compare_summary

{'rows_before': 8099,
 'rows_after': 8099,
 'total_missing_before': 3240,
 'total_missing_after': 0,
 'logical_violations_before': 1109,
 'logical_violations_after': 0,
 'numeric_max_after': {'tenure': 72.0,
  'MonthlyCharges': 200.0,
  'TotalCharges': 20000.0}}